In [96]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os

# Force reload the .env file
load_dotenv('.env', override=True)

print("DATABRICKS_HOST:", os.getenv("DATABRICKS_HOST"))
print("MLFLOW_TRACKING_URI:", os.getenv("MLFLOW_TRACKING_URI"))
print("MLFLOW_EXPERIMENT_ID:", os.getenv("MLFLOW_EXPERIMENT_ID"))
print("MLFLOW_REGISTRY_URI:", os.getenv("MLFLOW_REGISTRY_URI"))
print("DATABRICKS_TOKEN:", os.getenv("DATABRICKS_TOKEN"))


DATABRICKS_HOST: None
MLFLOW_TRACKING_URI: None
MLFLOW_EXPERIMENT_ID: None
MLFLOW_REGISTRY_URI: None
DATABRICKS_TOKEN: None


In [97]:
import mlflow
import mlflow.sklearn
import pandas as pd
import matplotlib.pyplot as plt

from numpy import savetxt

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [98]:
import hashlib

# Load data
# Define ordered column names to enforce consistent schema (overrides original file header)
columns = [
    'Date', 
    'Close',          
    'Open', 
    'High',                
    'Low', 
    'Volume'
]

btc_file = os.getenv("BTC_DATA")
btc_ocs_file=os.getenv("SENTIMENT_DATA")
# Read the raw CSV:
# - header=0 reads the first line (original names) but they are replaced by 'columns'
# - names=columns ensures consistent snake_case naming initialize above
btc_dt = (pd.read_csv(btc_file, header = 0, names = columns))
btc_ocs= (pd.read_excel(btc_ocs_file))

# Save processed data to a new CSV folder and appeand file name with hash value
    
# Create a hash of the DataFrame
df_hash = hashlib.md5(btc_dt.to_csv(index=False).encode()).hexdigest()
processed_btc_file = os.getenv("PROCESSED_BTC_DATA").replace(".csv", f"_{df_hash}.csv")
btc_dt.to_csv(processed_btc_file, index=False)



In [99]:
#Picking the useful columns from sentiment table
btc_ocs_extracted=btc_ocs[["Week_Code","Week_Start_Date","Week_End_Date","Sentiment"]]

In [100]:
#Modifying columns of the sentimemnt table for megring it with btc dataset
sentiment_dict={"Bullish":1,"Bearish":0}
btc_ocs_extracted["Sentiment"]=btc_ocs_extracted["Sentiment"].map(sentiment_dict)
btc_ocs_extracted["Week_Start_Date"]=pd.to_datetime(btc_ocs_extracted["Week_Start_Date"]) 
btc_ocs_extracted["Week_End_Date"]=pd.to_datetime(btc_ocs_extracted["Week_End_Date"]) 

C:\Users\kirti\AppData\Local\Temp\ipykernel_11436\3069174122.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_ocs_extracted["Sentiment"]=btc_ocs_extracted["Sentiment"].map(sentiment_dict)
C:\Users\kirti\AppData\Local\Temp\ipykernel_11436\3069174122.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_ocs_extracted["Week_Start_Date"]=pd.to_datetime(btc_ocs_extracted["Week_Start_Date"])
C:\Users\kirti\AppData\Local\Temp\ipykernel_11436\3069174122.py:5: SettingWithCopyWarning: 
A value is trying to b

In [101]:
#Modifying columns of the sentimemnt table for megring it with btc dataset
btc_dt["Date"]=pd.to_datetime(btc_dt["Date"]) 

In [103]:
btc_ocs_extracted_index= pd.IntervalIndex.from_arrays(btc_ocs_extracted["Week_Start_Date"], btc_ocs_extracted["Week_End_Date"], closed="both")
btc_dt["match_index"]=btc_ocs_extracted_index.get_indexer(btc_dt["Date"])
    

In [104]:
btc_dt["Sentiment"]=btc_dt["match_index"].map(btc_ocs_extracted["Sentiment"])
btc_dt["Week_Code"]=btc_dt["match_index"].map(btc_ocs_extracted["Week_Code"])
if "match_index" in btc_dt.columns:
    btc_dt.drop("match_index", axis=1,inplace=True)



In [105]:
btc_ocs_combined_file = os.getenv("COMBINED_DATA")
btc_dt.to_csv(btc_ocs_combined_file)
